In [1]:
#For a target protein, minimize and add in FAD and
#Inputs: 
    #model/models/tropb/ranked_0.pdb
    #cofactor/tropb/tropb_cdocker.pdb
    #cofactor/tropb/fad_in.pdb (from tropb_cdocker.pdb)
#Outputs:
    #cofactor/model_align/tropb.pdb
    #cofactor/pdb_with_fad/tropb_fad.pdb
import os
import sys

os.chdir('/home/azamh/demo/seq_struct_func/cofactor')

In [2]:
#Arguments
#Protein to add FAD cofactor to
model = 'tropb'
#Contains tropb_cdocker.pdb, a QM/MM refined structure that has 3mo docked with tropb using CDOCKER and structures of fad IN
tropbdir = './tropb'
#Contains rtf and parameter files for CHARMM
toppardir = '../toppar'
#Directory that contains AlphaFold structures
modeldir = '../model/models'
#Directory containing alphafold models after being aligned with tropb_cdocker.pdb
model_align_dir = './model_align'
#Directory with alphafold models and FAD, with cofactor taken from aligned tropb_cdocker.pdb
pdb_with_fad_dir = './pdb_with_fad'

#Structure to generate
pdb_with_fad = os.path.join(pdb_with_fad_dir, f'{model}_fad.pdb')

In [3]:
#Step 1: Align model with QM/MM refined structure of tropb docked with 3mo (could align with crystal structure as well)

#Generates model.psf and model.pdb in model_align directory
#Run TM align: superpose model with tropb structure
os.system(f'script/TMalign {modeldir}/{model}/ranked_0.pdb {tropbdir}/tropb_cdocker.pdb -o {model_align_dir}/{model}_sup')

#Run convpdb on superposed model so that pdb is in charmm readible format
os.system(f'convpdb.pl -out charmm22 -nohetero -segnames -nsel A: {model_align_dir}/{model}_sup.pdb > {model_align_dir}/{model.lower()}_conv.pdb')
model=model.lower() 


 *********************************************************************
 * TM-align (Version 20190822): protein structure alignment          *
 * References: Y Zhang, J Skolnick. Nucl Acids Res 33, 2302-9 (2005) *
 * Please email comments and suggestions to yangzhanglab@umich.edu   *
 *********************************************************************

Name of Chain_1: ranked_0.pdb (to be superimposed onto Chain_2)
Name of Chain_2: tropb_cdocker.pdb
Length of Chain_1: 447 residues
Length of Chain_2: 437 residues

Aligned length= 432, RMSD=   1.84, Seq_ID=n_identical/n_aligned= 0.958
TM-score= 0.92248 (if normalized by length of Chain_1, i.e., LN=447, d0=7.57)
TM-score= 0.94285 (if normalized by length of Chain_2, i.e., LN=437, d0=7.50)
(You should use TM-score normalized by length of the reference structure)

(":" denotes residue pairs of d <  5.0 Angstrom, "." denotes other aligned residues)
MPGSLIDTRQQPLSVGIVGGGIIGVILAAGLVRRGIDVKVFEQARGFREIGAGMAFTANAVRCMEMLDPAIVWALRSSGAVPISIGD-----

In [4]:
#Minimize model in vacuum to relax protein 
vacuum_min_out = os.popen(f'charmm -i script/initial_min.inp protein={model} modeldir={model_align_dir} toppardir={toppardir}').read()
print(vacuum_min_out)

#remove tm align and convpdb output
os.system(f'rm -rf {model_align_dir}/{model}_*')

[gollum154][[12721,1],0][../../../../../openmpi-3.1.2/opal/mca/btl/openib/btl_openib_component.c:1671:init_one_device] error obtaining device attributes for mlx5_0 errno says Protocol not supported
[gollum154.lsa.umich.edu:61167] [[12721,0],0] ORTE_ERROR_LOG: Data unpack would read past end of buffer in file ../../openmpi-3.1.2/orte/util/show_help.c at line 501


 Parameter: PROTEIN <- "TROPB"
 Parameter: MODELDIR <- "./MODEL_ALIGN"
 Parameter: TOPPARDIR <- "../TOPPAR"
1
                 Chemistry at HARvard Macromolecular Mechanics
           (CHARMM) - Developmental Version 45a2   February 15, 2020            
                             Git commit ID 51859fd                              
       Copyright(c) 1984-2014  President and Fellows of Harvard College
                              All Rights Reserved
  Current operating system: Linux-3.10.0-1127.el7.x86_64(x86_64)@gollum154.ls   
                 Created on 12/7/22 at 12:19:45 by user: azamh       

            Maximum number of ATOMS:    360720, and RESidues:      120240
 RDTITL> *VACUUM MINIMIZE INITIAL PROTEIN STRUCTURE
 RDTITL> *
  
 CHARMM>     
  
 CHARMM>    !read topology
 CHARMM>    read rtf card name @toppardir/top_all36_prot.rtf
 Parameter: TOPPARDIR -> "../TOPPAR"
 VOPEN> Attempting to open::../toppar/top_all36_prot.rtf::
 MAINIO> Residue topology file being read from uni

0

In [5]:
#Step 2: Place in FAD cofactor from aligned tropb structure (fad_in.pdb/fad_in.psf)
#Generates model_fad.pdb and model_fad.psf in models_w_fad directory  
fad_out = os.popen(f'charmm -i script/add_fad.inp protein={model} modeldir={model_align_dir} outdir={pdb_with_fad_dir} tropbdir={tropbdir} toppardir={toppardir}').read()
print(fad_out)

[gollum154][[12383,1],0][../../../../../openmpi-3.1.2/opal/mca/btl/openib/btl_openib_component.c:1671:init_one_device] error obtaining device attributes for mlx5_0 errno says Protocol not supported
[gollum154.lsa.umich.edu:61185] [[12383,0],0] ORTE_ERROR_LOG: Data unpack would read past end of buffer in file ../../openmpi-3.1.2/orte/util/show_help.c at line 501


 Parameter: PROTEIN <- "TROPB"
 Parameter: MODELDIR <- "./MODEL_ALIGN"
 Parameter: OUTDIR <- "./PDB_WITH_FAD"
 Parameter: TROPBDIR <- "./TROPB"
 Parameter: TOPPARDIR <- "../TOPPAR"
1
                 Chemistry at HARvard Macromolecular Mechanics
           (CHARMM) - Developmental Version 45a2   February 15, 2020            
                             Git commit ID 51859fd                              
       Copyright(c) 1984-2014  President and Fellows of Harvard College
                              All Rights Reserved
  Current operating system: Linux-3.10.0-1127.el7.x86_64(x86_64)@gollum154.ls   
                 Created on 12/7/22 at 12:19:51 by user: azamh       

            Maximum number of ATOMS:    360720, and RESidues:      120240
 RDTITL> *ADD FAD COFACTOR TO VACUUM MINIMIZED STRUCTURE
 RDTITL> *
  
 CHARMM>     
  
 CHARMM>    !read rtf
 CHARMM>    read rtf card name @toppardir/top_all36_prot.rtf
 Parameter: TOPPARDIR -> "../TOPPAR"
 VOPEN> Attempting to open::../toppa